In [28]:
import requests
import json
url = ('https://newsapi.org/v2/everything?'
       #'category=general&'
       'domains=bbc.co.uk&'
       #'country=gb&'
       'pageSize=99&'
       'page=1&'
       'from=2019-05-05&to=2019-06-03&'
       'apiKey=34e73062364d41b180a6f6a9625bff79')
response = requests.get(url)

with open('data/data.json', 'w') as outfile:
    json.dump(response.json(), outfile)

In [29]:
import pandas as pd

news_df = pd.DataFrame(columns=['Title', 'Date', 'Stems'])

In [30]:

with open("data/data.json") as f:
        loaded_json = json.load(f)
        for i in range(len(loaded_json["articles"])):
            if loaded_json["articles"][i]["content"] is not None:
                news_df = news_df.append({'Title': loaded_json["articles"][i]["title"], 'Date': loaded_json["articles"][i]["publishedAt"]}, ignore_index=True)
                #print(loaded_json["articles"][i]["title"])
                

KeyError: 'articles'

In [ ]:
news_df

## Stemming

In [ ]:
# import spacy
# from spacy import displacy
# from collections import Counter
# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=tokenizer.tokenize(sentence)
    token_words
    stop_words = set(stopwords.words('english'))
    #print(stop_words)
    filter_sentense = [w for w in token_words if not w in stop_words]
    
    stem_sentence=[]
    for word in filter_sentense:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
sentence=news_df.Title[0]
x=stemSentence(sentence)
print(sentence)
print(x)


In [ ]:
for i in news_df.index:
    news_df.Stems[i] = stemSentence(news_df.Title[i])

In [ ]:
wordfreq = []
master_wordlist = []
for i in news_df.index:
    wordlist = news_df.Stems[i].split()

    master_wordlist+= wordlist
#print(master_wordlist)
# print(set(master_wordlist))
# for w in master_wordlist:
#     wordfreq.append(master_wordlist.count(w))

# ww = list(zip(master_wordlist, wordfreq))
# print("Pairs\n" + str(ww[:]))

In [ ]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

In [ ]:
word_freq = wordListToFreqDict(master_wordlist)
# print(word_freq)

In [ ]:
# for x, y in word_freq.items():
#     print(x, y)
# print(word_freq.values())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook

output_notebook()

In [ ]:
fruits = list(word_freq.keys())
counts = list(word_freq.values())

# sorting the bars means sorting the range factors
sorted_fruits = sorted(fruits, key=lambda x: counts[fruits.index(x)])

p = figure(x_range=sorted_fruits, plot_height=500, plot_width=1400, title="Word frequencies",
           toolbar_location="below")

p.vbar(x=fruits, top=counts, width=0.9)
p.xaxis.major_label_orientation = "vertical"
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "25pt"
p.xaxis.major_label_text_font_size = "10pt"
p.y_range.start = 0

show(p)

## Google trends

In [2]:
import pandas as pd
from pytrends.request import TrendReq
import os  

In [15]:
if not (os.path.isfile('Data/crisis.pkl')):
    pytrend = TrendReq(hl='en-US, tz=360')

    kw_list = ['crisis']
    
    term_df = pytrend.get_historical_interest(kw_list, year_start=2015, year_end=2018)
    term_df.reset_index().to_pickle('Data/crisis.pkl')
else:
    term_df = pd.read_pickle('Data/crisis.pkl')
    print(term_df.head())
#fi

                 date  crisis  isPartial
0 2015-01-01 00:00:00      94      False
1 2015-01-01 01:00:00      82      False
2 2015-01-01 02:00:00      71      False
3 2015-01-01 03:00:00      72      False
4 2015-01-01 04:00:00      70      False


In [16]:
term_df.date.min(), term_df.date.max()

(Timestamp('2015-01-01 00:00:00'), Timestamp('2018-02-01 00:00:00'))

In [17]:
#add year and month
term_df['Year']  = term_df.date.apply(lambda x: x.year)
term_df['Month'] = term_df.date.apply(lambda x: x.month)
term_df.head()

,date,crisis,isPartial,Year,Month
0,2015-01-01 00:00:00,94,False,2015,1
1,2015-01-01 01:00:00,82,False,2015,1
2,2015-01-01 02:00:00,71,False,2015,1
3,2015-01-01 03:00:00,72,False,2015,1
4,2015-01-01 04:00:00,70,False,2015,1


In [21]:
temp_df = term_df.groupby(['Year']).mean().reset_index()
temp_df

,Year,crisis,isPartial,Month
0,2015,61.609964,False,6.526668
1,2016,64.192621,False,6.514033
2,2017,62.599864,False,6.526101
3,2018,67.740000,False,1.002667


In [18]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [27]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
p = figure(title="Year-wise total search of term crisis", y_axis_type="linear", plot_height = 400,
           tools = TOOLS, plot_width = 800)
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Total search'
#p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

p.line(temp_df.Year, temp_df.crisis,line_color="purple", line_width = 3)
# p.select_one(HoverTool).tooltips = [
#     ('year', '@x'),
#     ('Number of crimes', '@y'),
# ]

# output_file("line_chart.html", title="Line Chart")
show(p)